# パラメータ化 CHAPTER 5

本省の**パラメータ化 (parametrization)**では、1つのテスト関数を使って複数のテストケースを実行することで、より徹底的なテストをより少ない労力で行う方法を紹介する。

テストのパラメータ化は、テスト関数にパラメータを追加し、いくつかの引数をテストに渡すことで複数のテストケースを作成するというものである。ここでは、テストのパラメータ化を pytest で実装する方法として次の3つを取り上げる。パラメータ化の方法を選ぶときには、上から優先に考えることを推奨する。
- 関数をパラメータ化する
- フィクスチャをパラメータ化する
- `pytest_generate_tests` というフック関数を使う
この3つの方法を比較するため、同じパラメータ化問題をそれぞれの方法で解いてみよう。


## 5.1 パラメータ化せずにテストする
関数を使って何らかの値を送り込み、その出力が正しいかどうかをチェックするのは、ソフトウェアテストの一般的なパターンである。しかし、ひと組みの値を渡して関数を呼び出し、結果が正しいかどうかをチェックするだけでは、ほとんどの関数にとって十分なテストとは言えない。そこで、テストをパラメータ化し、何種類かのデータを用意して、同じテストを繰り返し呼び出す方法を考える。そのテストが1回でも失敗すれば、pytest はそのテストを失敗したと判定する。

まずは、cards プロジェクトの API の`finish()`メソッドのテストを見てみよう。

In [ ]:
# リスト5-1: cards_proj/src/cards/api.py
def finish (self, card_id: int):
    """Set a card state to 'done'."""
    self.update_card(card_id, Card(state="done"))

Cards アプリケーションにおけるカードの状態は"todo", "in prog", "done"
の3つであり、`finish()` メソッドはカードの状態を "done" に設定します。

このことをテストするために次のような方法で確認する: 
1. Card オブジェクトを作成してデータベースに追加し、 テストに使えるようにする。
2. finish () を呼び出す。
13 最終状態が"done"であることを確認する。

変数の1つは Card オブジェクトの開始時の状態である。
この変数には、"todo"、"prog"、最初から"done" のいずれかの値が設定される。
開始時の状態を3つともテストしてみよう。


In [ ]:
# リスト5-2:ch5/test_finish.py
from cards import Card

def test_finish_from_done (cards_db):
    index = cards.db.add_card(Card("write a book", state="done"))
    cards_db.finish(index)
    card = cards_db.get_card(index)
    assert card.state == "done"

def test_finish_from_todo(cards_db):
    index = cards_db.add_card(Card("create a course", state="todo"))
    cards_db.finish(index)
    card cards_db.get_card(index)
    assert card.state == "done"


これら3つのテスト関数は非常によく似ており、違いは状態 (state) とサマリー (summary) の値だけである。
実際にテストを実行結果は以下のとおりである。

実行結果貼り付け

上記の冗長性を減らす方法の1つとして、それらのコードを同じ関数にまとめることが考えられる。

In [ ]:
# UZ-5-3: ch5/test_finish_combined.py
from cards import Card
def test_finish(cards_db):
    for c in [
        Card ("write a book", state="done"),
        Card ("second edition", state="in prog"),
        Card("create a course"
        state="todo")
    ]:
    書き写し

実行結果は以下の通り

In [ ]:
実行結果貼り付け

テストは成功し、一見冗長性も軽減できている。しかし、以下のような問題が残ってしまっている。
- 報告されるテストケースが3つではなく1つになっている。
- テストケースの1つが失敗した場合、トレースバックか他のデバッグ情報を調べない限り、どのテストが失敗したのかわからない。
- テストケースの1つが失敗した場合、その後のテストケースは実行されない。pytest は assert が失敗した時点でテストの実行を中止する。

pytest には、この種の問題を解決するためのパラメータ化に関する機能が十分に備わっている。
ここでは、関数のパラメータ化、フィクスチャのパラメータ化、`pytest_generate_tests`の順に見ていこう。


## 5.2 関数をパラメータ化する
テスト関数をパラメータ化するには、テストの定義にパラメータを追加し、テストに渡す引数を定義する。
引数の定義には、@pytest.mark.parametrize() マーカーを使う。

In [ ]:
# リスト5-4:ch5/test_func_param.py


`test_finish` 関数には、元のパラメータである `cards_db` フィクスチャに加えて、`start_summary` と `start_state` の2つが追加されている。これらのパラメータは`@pytest.mark.parametrize()` の1つ目の引数と一致する。 

`@pytest.mark.parametrize()`の1つ目の引数は、パラメータの名前のリストである。これらの名前は文字列であり、["start_summary", "start_state"] のように文字列のリストとして指定するか、"start_summary, start_state" のようにコンマ区切りの文字列として指定する。

`@pytest.mark.parametrize()`の2つ目の引数は、テストケースのリストである。このリストの各要素はタプルまたはリストとして表されたテストケースであり、テスト関数に渡される引数ごとに1つの要素を含んでいる。

pytest は、 このテストを (start_summary, start_state) ペアごとに実行し、別々のテストとして報告する。


実行結果

この `parametrize()` は狙いどおりの働きをしたようです。しかし、テストケースごとに `summary` を変更することはこのテストにとってあまり重要ではなく、意味もなく複雑になるだけである。そこでパラメータ化を `start_state` だけにしてみよう。

In [ ]:
# リスト5-5ch5/test_func_param.py


テストの大部分は以前と同じであり、パラメータの「リスト」に含まれているのは1つ　("start_state") である。テストケースのリストに含まれているパラメータの値も1つだけになっています。

`start_summary`パラメータはもう関数の定義に含まれていません。`start_summary` は `Card()` 呼び出しにハードコーディングされている。

このテストを実行すると、肝心の変更箇所に焦点が絞られることがわかる。


実行結果

2つの例の出力を見比べてみると、この例で表示されているのが `start_state` パラメー夕の値 ("todo", "in prog", "done") だけであることがわかる。
最初の例では pytest が両方のパラメータの値をハイフン(-)で区切って表示されていたが、変化するパラメータが1つだけになったためハイフンは必要なくなっている。

テストコードでも出力でも `start_state` の違いに焦点が絞られている。 テストコードでの違いは些細なものなので、筆者はつい必要以上にパラメータを追加してしまう。しかし、出力での違いは歴然で、テストケースの違いが出力にはっきり表れます。
こういった出力の明確さはテストケースが失敗したときに大きな助けになる。
テストの失敗にとって意味を持つ変更箇所にすぐに目星を付けることができます。

本節では関数のパラメータ化の仕方を見てきたが、次節のようにフィクスチャのパラメータ化を使って同じテストを書くこともできる。

## 5.3 フィクスチャをパラメータ化する
関数をパラメータ化したときには、指定した引数セットごとに pytest がテスト関数を1回呼び出した。
本節でのフィクスチャのパラメータ化では、それらのパラメータをフィクスチャに移動する。そのようにすると、指定した値セットごとに pytest がフィクスチャを 1回呼び出すようになる。そして、そのフィクスチャに依存しているすべてのテスト関数がフィクスチャの値ごとに1回呼び出される。

In [ ]:
# リスト5-6:ch5/test.fix.param.py


`start_state()` は、`params` の値ごとに1回、合計3回呼び出される。`params` の値がそれぞれ`request.param`に保存され、フィクスチャによって使われる。パラメータ値に依存するコードを `start_state()` の中に配置することも可能。 ただし、この場合は単にパラメータの値を返している。
`test_finish()` は、関数のパラメータ化で使った `test_finish_simple()` とまったく同じですが、`parametrize()` マーカーは付いていない。この関数は `start_state` をパラメータとして使っているため、`start_state()` フィクスチャに渡される値ごとに pytest によって1回呼び出される。テストを実行すると、前節と同じ出力が生成される。


実行結果

一見、フィクスチャのパラメータ化の目的は関数のパラメータ化と同じで、コードが少し増えるだけのようにも見えるが、状況によっては、フィクスチャのパラメー夕化のほうに分があることがある。
フィクスチャのパラメータ化の利点は、引数セットごとにフィクスチャを実行できることであり、テストごとに実行するセットアップやティアダウンのコードがある場合に役立つ。

たとえば、異なるデータベースに接続したり、 内容が異なるファイルを選んだりできる。

また、フィクスチャのパラメータ化には、同じパラメータセットで多くのテスト関数を実行できるという利点もある。
`start_state`フィクスチャを使っているテストはすべて、パラメータの値ごとに1回、合計3回呼び出されるようになる。

フィクスチャのパラメータ化は、同じ問題を別の角度から捉える方法でもある。「同じテスト、異なるデータ」の観点から考えた場合、`finish()` のテストであっても、関数のパラメータ化のほうを選ぶのが良いだろう。
しかし、「同じテスト、異なる `start_state`」の観点から考えた場合は、フィクスチャのパラメータ化のほうを選ぶのが良いだろう。

## 5.4 pytest_generate_tests を使ってパラメータ化する

テストをパラメータ化する3つ目の方法は、`pytest-generate_tests`というフック関数を使うことである。フック関数は pytest の通常の処理フローを変更するためにプラグインでよく使われる。しかし、フック関数の多くはテストファイルやconftest.py ファイルで使うことができる。

`pytest_generate_tests` を使って前節と同じフローを実装すると、次のように書くことができる。

In [ ]:
# リスト5-7:ch5/test_gen.py

`test_finish()`は以前のものと同じである。変更したのは、テストが呼び出されるたびに pytest が `start_state` の値を設定する方法だけである。

この `pytestgenerate_tests`関数は、実行するテストのリストを組み立てるときに pytest によって呼び出される。`metafunc`オブジェクトはさまざまな情報を含んでいるが、ここでは単にパラメータ名の取得とパラメータの生成に使っている。

このテストを実行すると、見覚えのある出力が表示される。


実行結果

実際には、`pytest-generate_tests` 関数は途轍もなく強力です。 この単純な例では、 先の2つのパラメータ化と同じことをするだけですが、テストの収集時にパラメータリス トをおもしろい方法で変更したい場合は、 この関数が大きな助けになります。

たとえば、`pytest-generate_tests` 関数を使って次のようなことができる。

1. metafunc を使って metafunc.config.getoption("--someflag")にアクセスできるため、コマンドラインフラグに基づいてパラメータリストを作成できる。 さらに多くの値をテストするために--excessive フラグを追加したり、一部の値 だけをテストするために--quick フラグを追加したりできる。
2. 別のパラメータの有無に基づいてパラメータリストを作成できる。たとえば、関 連する2つのパラメータを要求するテスト関数では、パラメータを1つだけ要求 する場合とは異なる値を使って両方のパラメータを設定できる。
3. 関連する2つのパラメータをたとえば次のようにして同時にパラメータ化できる。


テストをパラメータ化する3つの方法を見てきましたが、ここでは単に finish()に対する1つのテスト関数から3つのテストケースを作成するためにパラメータ化を使いました。

しかし、パラメータ化を利用すれば、大量のテストケースを生成することも考えられます。

そこで次節では、-k フラグを使ってテストの一部を選択する方法を紹介します。



## 5.5 キーワードを使ってテストケースを選択する
大量のテストケースをすばやく作成することにかけては、パラメータ化の威力は絶大です。このため、テストの一部だけを実行できると便利なことがよくあります（-k フラグは第2章の2.9節が初出）。本章ではテストケースの数が多いため、 このオプションを試してみることにしましょう。

出力結果

"play" または "create" を含んでいるテストケースを除外したい場合は、さらに絞り込
むことができます。


出力結果

テスト関数を1つだけ選択することもできます。 そのテスト関数はすべてのパラメータ を使って実行されます。

出力結果

テストケースを1つだけ選択することもできます。

出力結果

うれしいことに、テストの一部を選択する一般的な方法はすべてパラメータ化されたテストでも使うことができます。
これらの手法は新しいものではありませんが、筆者はバラメータ化されたテストを実行したりデバッグしたりするときによく使っています。

※ 引用符を使う
ハイフン(-)、角かっこ([])、スペースをそのまま使うとコマンドシェルに 渉することになります。 パラメータ化されたテストを選択するときには、引用符 (") を使うようにしてください。